# Pydeck example NAV permitteringer
> An example of fetching data from excel and visualizing the data with pydeck.

In [40]:
import deetly
import pandas as pd
import geopandas as gpd
import pydeck
import altair as alt
import dateutil.parser
import requests
import json
import os
from IPython.display import IFrame

'no_NO.UTF-8'

In [44]:
MAPBOX_TOKEN = os.environ['MAPBOX_TOKEN'] 

In [48]:
## workaround to show figure in jupyter-book
def show_fig(fig, name):
    if get_ipython().__class__.__name__ == 'ZMQInteractiveShell':
        print('Jupyter')
        res = fig.to_html(name)
    else:
        print('Not Jupyter')
        res = fig.to_html(name, notebook_display=False)
        IFrame(src=name, width=700, height=600) 

In [52]:
# Create datapackage
description = """
## NAV: Søknader om dagpenger

Siden torsdag 12. mars, dagen mange av de mest omfattende tiltakene ble innført for å begrense smittespredningen, er det kommet inn totalt 274 200 søknader om dagpenger, hvorav 250 800 gjelder dagpenger ved permittering.

Så langt i 2020 er det kommet inn totalt 313 100 søknader om dagpenger, sammenlignet med 160 500 gjennom hele 2019 og 166 600 gjennom hele 2018.
"""

package = {
    "name":"NAV: Søknader om dagpenger", 
    "description":description, 
    "author":"deetly", 
    "theme":"example",
    "license": "MIT",
    "keywords": ["example, deck.gl, pydeck, arbeid, dagpenger, permittering, permitteringer"]
}

dp = deetly.package(package)

Missing mandatory props(s): ['title']
Missing recommended props(s): ['contactPoint', 'distribution', 'keyword', 'publisher', 'spatial', 'temporal']
Property not in DCAT schema: ['name', 'author', 'license', 'keywords']
Invalid type: ['String:name', 'String:description', 'String:author', 'String or list of strings:theme', 'String:license', 'String:keywords', 'ISO date or datetime:issued', 'ISO date or datetime:modified']


# Get data from nav.no

In [ ]:
EXCEL_URL_NÆRING = 'https://www.nav.no/no/nav-og-samfunn/statistikk/flere-statistikkomrader/relatert-informasjon/soknader-om-dagpenger/_/attachment/download/45180f2d-4445-4f83-834e-5d8c95d2bf90:120845241b1ca3a76ab8d6b47559f543a743e4f8/2503%20-%20S%C3%B8knader_om_dagpenger._N%C3%A6ring.xlsx'

In [59]:
EXCEL_URL = 'https://www.nav.no/no/nav-og-samfunn/statistikk/flere-statistikkomrader/relatert-informasjon/soknader-om-dagpenger/_/attachment/download/78bbd22d-8205-4353-bfd1-dba90f8b6f99:252d4b00554bb9851e3a42e4681ccdbcfcb0fe41/2603%20-%20S%C3%B8knader_om_dagpenger.xlsx'

In [ ]:
MAP_URL = 'https://raw.githubusercontent.com/datasett/maps/master/norway/data/2020/counties_2020_s_geojson.json'

In [66]:
xl_permittering = pd.ExcelFile(EXCEL_URL)
xl_permittering.sheet_names

['Dato',
 'Uke',
 'Fylke. Ordinære',
 'Fylke. Permitterte',
 'Kommune. Permitterte',
 'Bydel. Permitterte',
 'År',
 'Om statistikken']

In [70]:
df_p = pd.read_excel(EXCEL_URL,sheet_name='Kommune. Permitterte', header=6)
df_p.drop(columns=["Unnamed: 0","Unnamed: 2","Unnamed: 7","Unnamed: 13","Unnamed: 14"], axis=1, inplace=True)
df_p.rename(columns={"Antall søknader": "Kommune"}, inplace=True)
df_p.drop([0, 0],inplace=True)
df_p.replace({'Kommune': {'Ukjent': '9999 Ukjent'}}, inplace=True)
df_p.replace({'*':0}, inplace=True)
df_p["kommunenummer"] = df_p["Kommune"].apply(lambda x: x[0:4])
df_p["kommune_navn"] = df_p["Kommune"].apply(lambda x: x[4:])
df_p.rename(columns={"Kommune": "kommune", "I alt": "sum", 11: 'uke11', 12: 'uke12', 13:'uke13'}, inplace=True)
df_p.rename(columns={"I alt.1": "sum_andel", '11.1': 'uke11_andel', '12.1': 'uke12_andel', '13.1':'uke13_andel'}, inplace=True)
df_p['sum_andel'] = df_p['sum_andel'].apply(lambda x: round(x,2))
df_p['uke11_andel'] = df_p['uke11_andel'].apply(lambda x: round(x,1)) 
df_p['uke12_andel'] = df_p['uke12_andel'].apply(lambda x: round(x,1)) 
df_p['uke13_andel'] = df_p['uke13_andel'].apply(lambda x: round(x,1)) 
df_p.drop(columns=['Andel av arbeidsstyrke'], inplace=True)
df_p.head(1)

,kommune,sum,uke11,uke12,uke13,sum_andel,uke11_andel,uke12_andel,uke13_andel,kommunenummer,kommune_navn
1,0301 Oslo,40504,3976,29746,6782,10.42,1.0,7.7,1.7,0301,Oslo


In [74]:
df_p.sort_values('sum', ascending=False).head(1)

,kommune,sum,uke11,uke12,uke13,sum_andel,uke11_andel,uke12_andel,uke13_andel,kommunenummer,kommune_navn
1,0301 Oslo,40504,3976,29746,6782,10.42,1.0,7.7,1.7,0301,Oslo


In [78]:
df_map = gpd.read_file(MAP_URL)
df_merged = pd.merge(df_map, df_p, how='left', on='kommunenummer')
df_merged['lat'] = df_merged['geometry'].apply(lambda x : round(x.centroid.y,2))
df_merged['lng'] = df_merged['geometry'].apply(lambda x : round(x.centroid.x,2))
df_merged['coordinates']  = df_merged.apply(lambda row: [row['lng'], row['lat']], axis=1)

In [82]:
df_merged.head(1)

,id,kommunenummer,oppdateringsdato,navn,area_kommune,fylke_nummer,fylke_navn,geometry,kommune,sum,...,uke12,uke13,sum_andel,uke11_andel,uke12_andel,uke13_andel,kommune_navn,lat,lng,coordinates
0,0,0301,2019-10-28T00:00:00,Oslo,0.072515,03,OSLO,"POLYGON ((10.68032 60.13353, 10.74694 60.12303...",0301 Oslo,40504,...,29746,6782,10.42,1.0,7.7,1.7,Oslo,59.98,10.74,"[10.74, 59.98]"


In [86]:
view_state_norway = pydeck.ViewState(
    longitude = 17,
    latitude= 64,
    zoom= 3,
    minZoom= 3,
    maxZoom= 12,
    pitch= 45,
    bearing= 0)

In [90]:
tooltip = {
    "html": "<b>{navn}</b> {sum}",
    "style": {
        "backgroundColor": "silver",
        "color": "black",
        "zIndex": "10000",
    },
}

## Scatterplot

In [94]:
source=df_merged[['navn','fylke_navn','sum','sum_andel','coordinates','lat','lng']]
source.head(1)

,navn,fylke_navn,sum,sum_andel,coordinates,lat,lng
0,Oslo,OSLO,40504,10.42,"[10.74, 59.98]",59.98,10.74


In [ ]:
view_state = pydeck.ViewState(
    longitude = 17,
    latitude= 64,
    zoom= 3,
    minZoom= 3,
    maxZoom= 12,
    pitch= 0,
    bearing= 0)

layer = pydeck.Layer(
    "ScatterplotLayer",
    data=source,
    pickable=True,
    opacity=0.8,
    stroked=True,
    filled=True,
    radius_scale=2,
    radius_min_pixels=1,
    radius_max_pixels=200,
    line_width_min_pixels=1,
    get_position=["lng", "lat"],
    get_radius="sum",
    get_fill_color=[255, 140, 0, 200],
    get_line_color=[0, 0, 0],
)

fig = pydeck.Deck(layers=[layer], tooltip=tooltip, initial_view_state=view_state, mapbox_key=MAPBOX_TOKEN)
show_fig(fig, 'fig.html')

In [158]:
fig = pydeck.Deck(layers=[layer], tooltip=tooltip, initial_view_state=view_state, mapbox_key=MAPBOX_TOKEN)
res = fig.to_html('test.html', notebook_display=False)
IFrame(src='test.html', width="700", height="600") 

In [108]:
# Add figure to data package
description = """
## Sum antall søknader om dagpenger ved permittering.
"""

dp.pydeck(fig, "Sum antall",description, showMap=True,tooltip=json.dumps(tooltip))

AttributeError: 'dict' object has no attribute 'to_json'

## Column layer

In [ ]:
layer = pydeck.Layer(
    "ColumnLayer",
    source,
    get_position=["lng", "lat"],
    get_elevation="sum_andel",
    elevation_scale=10000,
    radius=5000,
    get_fill_color='[sum_andel*10, 140, 0, 200]',
    pickable=True,
    auto_highlight=True,
)

tooltip = {
    "html": "<b>{navn}</b><br/>Sum antall: {sum}</br>Andel: {sum_andel}%",
    "style": {
        "backgroundColor": "silver",
        "color": "black",
        "zIndex": "10000",
    },
}

fig = pydeck.Deck(
    layer,
    initial_view_state=view_state_norway,
    tooltip=tooltip,
    mapbox_key=MAPBOX_TOKEN
)

show_fig(fig, 'fig2.html')

In [118]:
# Add figure to data package
description = """
## Sum antall søknader om dagpenger ved permittering som andel av arbeidsstyrken
"""

dp.pydeck(fig, "Andel av arbeidsstyrken",description, showMap=True, tooltip=json.dumps(tooltip))

AttributeError: 'dict' object has no attribute 'to_json'

## Utvikling antall pr uke & kommune

In [ ]:
source = df_p[['kommune', 'uke11', 'uke12', 'uke13']]
source = source.melt(id_vars='kommune')

In [128]:
fig = alt.Chart(source).mark_bar().encode(
    x=alt.X('sum(value):Q',axis=alt.Axis(title='')),
    y=alt.Y('variable:N',axis=alt.Axis(title='')),
    color=alt.Color('kommune:N',legend=None),
    order=alt.Order(
      'value',
      sort='descending'
    ),
    tooltip=[alt.Tooltip('kommune', title='Kommune'),
             alt.Tooltip('variable', title='Uke'),
             alt.Tooltip('sum(value)', title='Antall')]
)

fig

alt.Chart(...)

In [132]:
description = """
## Sum antall søknader om dagpenger ved permittering

"""

dp.vega(fig,"Antall pr uke og kommune", description)

## Utvikling andel pr uke & kommune

In [135]:
source = df_p[['kommune', 'uke11_andel', 'uke12_andel', 'uke13_andel']]
source = source.melt(id_vars='kommune')

In [138]:
fig = alt.Chart(source).mark_bar().encode(
    x=alt.X('sum(value):Q',axis=alt.Axis(title='')),
    y=alt.Y('variable:N',axis=alt.Axis(title='')),
    color=alt.Color('kommune:N',legend=None),
    order=alt.Order(
      'value',
      sort='descending'
    ),
    tooltip=[alt.Tooltip('kommune', title='Kommune'),
             alt.Tooltip('variable', title='Uke'),
             alt.Tooltip('sum(value)', title='Antall')]
)

fig

ValueError: kommune encoding field is specified without a type; the type cannot be inferred because it does not match any column in the data.

In [142]:
description = """
## Sum antall søknader om dagpenger ved permittering som andel av arbeidsstyrken

"""

dp.vega(fig,"Andel pr uke og kommune", description)

# Næringer

In [144]:
xl_n = pd.ExcelFile(EXCEL_URL_NÆRING)
xl_n.sheet_names

['Om statistikken',
 'Forklaring tall',
 'Næringsgruppe. Søknad permitter',
 'Næringsgruppe pr fylke.Søknad p',
 'Næringsgruppe. Andel. Søknad pe',
 'Næring. Søknad permitterte',
 'Næringsgruppe. Søknad ordinære']

## Næringer liste

In [145]:
df_n = pd.read_excel(EXCEL_URL_NÆRING,sheet_name='Næring. Søknad permitterte', header=11)
df_n.drop(columns=["Unnamed: 0","Unnamed: 3","Unnamed: 6","Unnamed: 7","Unnamed: 8"], axis=1, inplace=True)
df_n.rename(columns={157: "personer", 106: "virksomheter", '00': "næring_kode",'Uoppgitt': "næring"}, inplace=True)
df_n['label'] = df_n['næring']

In [146]:
df_n.head(1)

,næring_kode,næring,personer,virksomheter,label
0,01,"Jordbruk og tjenester tilknyttet jordbruk, jak...",1203,618,"Jordbruk og tjenester tilknyttet jordbruk, jak..."


In [147]:
df_n.iloc[1].label

'Skogbruk og tjenester tilknyttet skogbruk'

In [148]:
source = df_n[['label','personer']].copy()
source.columns=['name','value']
source['value'] = source['value'].apply(lambda x: int(str(x).replace('*','0')))
source['name'] = source['name'] + ':' + source['value'].astype(str)
source.head(1)

,name,value
0,"Jordbruk og tjenester tilknyttet jordbruk, jak...",1203


In [149]:
data = source.to_json(orient='records')

columns = [
  {
    "name": "Næring",
    "selector": "name",
    "sortable": True,
  },
  {
    "name": "Personer",
    "selector": "value",
    "sortable": True,
  },
]

fig = {
  "data": json.loads(data),
  "layout": {
    "title": "Søknad om dagpenger ved permittering. Fra uke 11",
    "columns": columns,
    "sortable": True,
  }
}

In [150]:
# Add barlist figure to the datapackage
description = """
## Søknad om dagpenger ved permittering. Fra uke 11
"""

dp.barlist(fig,"Søknader om dagpenger ved permittering", description)

## Næringsgruppe andel

In [151]:
df_n = pd.read_excel(EXCEL_URL_NÆRING,sheet_name='Næringsgruppe. Andel. Søknad pe', header=6)
df_n.drop(columns=["Unnamed: 0","Unnamed: 2","Unnamed: 14"], axis=1, inplace=True)
df_n.rename(columns={"Unnamed: 1": "Næring"}, inplace=True)

In [152]:
df_n.head(1)

,Næring,Oslo,Rogaland,Møre og Romsdal,Nordland,Viken,Innlandet,Vestfold og Telemark,Agder,Vestland,Trøndelag - Trööndelage,Troms og Finnmark - Romsa ja Finnmárku
0,"Jordbruk, skogbruk og fiske",0.121212,0.0558,0.036242,0.040583,0.076695,0.07328,0.054682,0.07211,0.063291,0.056325,0.048407


In [153]:
source=df_n.melt(id_vars="Næring")
source['Fylke'] = source['variable'].apply(lambda x: x.split('-')[0])
source['Prosent'] = source['value'].apply(lambda x: round(100*x,1))

In [154]:
fig = alt.Chart(source).mark_rect().encode(
    x = alt.X('Næring:O',axis=alt.Axis(title='')),
    y = alt.Y('Fylke:O',axis=alt.Axis(title='')),
    color=alt.Color('Prosent:Q',legend=alt.Legend(orient="top")),
    tooltip=[alt.Tooltip('Næring', title='Næring'),
             alt.Tooltip('Fylke', title='Fylke'),
             alt.Tooltip('Prosent', title='Prosent')]
)

fig    

alt.Chart(...)

In [155]:
description = """
Fra uke 11. Prosentandel av lønnstakere

"""

dp.vega(fig,"Søknader om dagpenger. Andel/Næring", description)

## Publish

In [156]:
dp.publish()

[]
Public datapackage: https://storage.googleapis.com/deetly/examples/15f6894f876c84cdb223ff534a6d3292/datapackage.json 

Published at: https://public.deetly.com/examples/15f6894f876c84cdb223ff534a6d3292 

